In [54]:
f = open("input.txt", "r")

seeds = []
headers = ["seed-to-soil", "soil-to-fertilizer", "fertilizer-to-water", "water-to-light", "light-to-temperature", "temperature-to-humidity", "humidity-to-location"]
maps = [[] for i in range(len(headers))]

lines = []
for line in f:
    lines.append(line.replace("\n", ""))

# Parse the input to create maps
i = 0
header_index = 0
while i < len(lines):
    line = lines[i]
    if line.startswith("seeds:"):
        seeds = [int(number) for number in line.split(":")[1].split(" ") if number != ""]
        i += 1
    elif line.startswith(headers[header_index]):
        i += 1
        while i < len(lines) and lines[i] != "": 
            line = lines[i]
            maps[header_index].append([int(number) for number in line.split(" ") if number != ""])
            i += 1
        header_index += 1
    else:
        i += 1

# For each seed, go through all maps to get the corresponding location
locations = []
for seed in seeds:
    current_value = seed
    for i in range(len(headers)):
        for [dest_start, source_start, range_len] in maps[i]:
            dist_to_start = current_value - source_start
            if dist_to_start >= 0 and dist_to_start < range_len:
                # Value is inside range, convert it to its destination and skip to next map
                current_value = dest_start + dist_to_start
                break
    locations.append(current_value)

print(min(locations))


324724204


In [55]:
# Going the brute force way would be way too long given the numbers in play.
# Define a function that takes an interval and returns the list of intervals we get after using a map to convert values
def translate(interval, map):
    result = []
    current_intervals = []
    remaining_intervals = [interval]
    # At then end of each loop, remaning_intervals represents the values that haven't been converted yet, as a list of intervals.
    # This is to account for the fact that a given interval could have only part of its values converted by a line of the map.
    # Then the remaning values could be split in two non adjacent intervals, that need to go through the next lines of the map
    # to check if they need to be converted or not.
    for [dest_start, source_start, range_len] in map:
        # Go to next line of the map to convert the intervals that haven't been converted yet
        current_intervals = remaining_intervals
        remaining_intervals = []
        while len(current_intervals) > 0:
            # Treat intervals one at a time, removing them from the list
            interval = current_intervals.pop()
            interval_start, interval_range = interval
            interval_end = interval_start + interval_range
            source_end = source_start + range_len
            s_s_diff = interval_start - source_start
            s_e_diff = interval_start - source_end
            e_e_diff = interval_end - source_end
            e_s_diff = interval_end - source_start
            if (s_s_diff < 0 and e_s_diff < 0) or (s_e_diff > 0):
                # There is no overlap between the interval and the source range of this line of the map
                remaining_intervals.append(interval)
                continue
            else:
                # There is an overlap, we need to find its start and end
                overlap_start = max(interval_start, source_start)
                overlap_end = min(interval_end, source_end)
                dest_overlap_start = dest_start + overlap_start - source_start
                # The overlap is converted to its equivalent interval in destination space, using the map, then added to the final result
                result.append([dest_overlap_start, overlap_end - overlap_start])
                if s_s_diff < 0:
                    # The leftmost values of the original interval haven't been converted because there were outside the range of the map line
                    # so we need to add them to the intervals to be treated by the rest of the map
                    remaining_interval_start = interval_start
                    remaining_interval_end = overlap_start
                    remaining_intervals.append([remaining_interval_start, remaining_interval_end - remaining_interval_start])
                if e_e_diff > 0:
                    # The rightmost values of the original interval haven't been converted because there were outside the range of the map line
                    # so we need to add them to the intervals to be treated by the rest of the map
                    remaining_interval_start = overlap_end
                    remaining_interval_end = interval_end
                    remaining_intervals.append([remaining_interval_start, remaining_interval_end - remaining_interval_start])
    for interval in remaining_intervals:
        # The remaning intervals are source values that haven't been converted by any map line. Their values in the destination space are 
        # the same as in the source space so we can just add these intervals to the result
        result.append(interval)
    return result
            
seed_index = 0
# Holds the final result: the list of location intervals after going through all the maps
location_intervals = []
# We need to go through all our seeds intervals
while seed_index < len(seeds):
    seed_start = seeds[seed_index]
    seed_range = seeds[seed_index+1]
    seed_index += 2
    # One interval can generate several destination intervals after going through a map so we work on list of intervals
    seed_interval = [seed_start, seed_range]
    intervals = [seed_interval]
    # Looping through all maps
    for i in range(len(headers)):
        result = []
        # Each interval gets translated through the current map and added to the result
        for interval in intervals:
            result += translate(interval, maps[i])
        # The resulting intervals will go through the next map
        intervals = result
    # After going through all the maps, the resulting intervals are the location intervals we were looking for
    location_intervals += intervals
print(location_intervals)

# We want to find the minimum location value so we just check each interval to find which one starts at the lowest value
min_location = location_intervals[0][0]
for i in range(1, len(location_intervals)):
    min_location = min(min_location, location_intervals[i][0])

# The final answer!
print(min_location)



[[1414626613, 8743318], [2007321506, 250171], [1132190688, 8323030], [1824642538, 45479686], [104899384, 9358577], [4034910812, 12542279], [1728984261, 2960365], [1998006408, 9315098], [349894056, 20854432], [1387021080, 6554759], [1332910116, 10833852], [2084274373, 30242895], [2011729910, 6573175], [1140513718, 1513232], [104070862, 828522], [2125982290, 22654187], [2495852942, 504707], [2608413895, 29582294], [3453395265, 28349171], [3228866466, 14955980], [3243822446, 24333031], [1978366280, 392287], [1716132415, 7792395], [1978758567, 19247841], [595027566, 38258929], [559945395, 25332656], [516299014, 7717806], [2687906049, 15580427], [1731944626, 22763517], [2894939130, 22257018], [3170145204, 11076107], [146071405, 28896049], [174967454, 97134270], [3985313388, 3784155], [3989097543, 910639], [1343743968, 5980660], [2987683790, 53337395], [3344051857, 5203871], [1924451656, 2786414], [3363570267, 379212], [3667533047, 110883165], [3778416212, 25755942], [2539591784, 13139645], 